In [ ]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
import ollama

In [ ]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for a Windows Operation System. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [ ]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary packages such as iomanip,chrono.\n\n"
    user_prompt += python
    return user_prompt

In [ ]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [ ]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized_codellama.cpp", "w") as f:
        f.write(code)

"""
def write_output(cpp_block):
    import re

    # Match the first ```cpp ... ``` block
    match = re.search(r"```(?:cpp)?\s*(.*?)```", cpp_block, re.DOTALL)
    if match:
        code = match.group(1)
        with open("optimized_codellama.cpp", "w") as f:
            f.write(code)
        print("✅ C++ code written to optimized_codellama.cpp")
    else:
        print("⚠️ No valid ```cpp code block found.")
"""

In [ ]:
#This is with streaming 
import ollama
def optimize_codellama(python):
    stream = ollama.chat(
        model='codellama:latest',
        #model='deepseek-coder:latest',
        #model='codeqwen:latest',
        #model='starcoder2:latest',
        messages=messages_for(python),
        stream=True
    )
    reply = ""
    for chunk in stream:
        fragment = chunk['message']['content'] or ""
        reply+=fragment
        print(fragment, end='', flush=True)
        #yield result
    write_output(reply)

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
exec(pi)

In [ ]:
optimize_codellama(pi)

In [ ]:
exec(pi)

In [ ]:
!g++ -O3 -std=c++17 -o optimized_codellama.exe optimized_codellama.cpp
!optimized_codellama.exe

In [ ]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
exec(python_hard)

In [ ]:
optimize_codellama(python_hard)

In [ ]:
!g++ -O3 -std=c++17 -o optimized_codellama.exe optimized_codellama.cpp
!optimized_codellama.exe

In [ ]:
def stream_codellama(python):    
    stream = ollama.chat(model='codellama:latest', messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk['message']['content'] or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')
    """
    stream = ollama.chat(model='codellama:latest', messages=messages_for(python), stream=True)
    # No need for 'reply' variable to accumulate the full text here
    for chunk in stream:
        fragment = chunk['message']['content'] or ""
        # Apply replacements to the fragment before yielding
        processed_fragment = fragment.replace('```cpp\n', '').replace('```', '')
        yield processed_fragment # <--- YIELD ONLY THE NEW FRAGMENT
    """

In [ ]:
def stream_codeqwen(python):    
    stream = ollama.chat(model='codeqwen:latest', messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk['message']['content'] or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')
    """
    stream = ollama.chat(model='codeqwen:latest', messages=messages_for(python), stream=True)
    for chunk in stream:
        fragment = chunk['message']['content'] or ""
        processed_fragment = fragment.replace('```cpp\n', '').replace('```', '')
        yield processed_fragment
    """ 

In [ ]:
#import time
def optimize(python, model):
    #print(f"User selected model: {model}")
    time.sleep(10)
    if model=="codellama:latest":
        result = stream_codellama(python)
    elif model=="codeqwen:latest":
        result = stream_codeqwen(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far      

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["codellama:latest", "codeqwen:latest"], label="Select model", value="codellama:latest")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

In [ ]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [ ]:

def write_output1(cpp,model):
    code = cpp.replace("```cpp","").replace("```","")
    safe_model_name = model.replace(":", "_")
    filename = f"{safe_model_name}.cpp"
    with open(filename, "w") as f:
        f.write(code)
    return filename

In [ ]:
def execute_cpp(code, model):
    #write_output1(code,model)
    filename=write_output1(code,model)
    try:
        #compile_cmd = ["!g++", "-O3", "-std=c++17", "","","","-o", "optimized", "optimized.cpp"]
        #compile_cmd = ["g++", "-O3", "-std=c++17", "optimized.cpp", "-o", "optimized"]
        compile_cmd = ["g++", "-O3", "-std=c++17", filename, "-o", "optimized"]
        compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
        #run_cmd = ["!optimized"]
        run_cmd = ["./optimized"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

#!g++ -O3 -std=c++17 -o optimized_codellama.exe optimized_codellama.cpp
#!optimized_codellama.exe

In [ ]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["codellama:latest", "codeqwen:latest"], label="Select model", value="codeqwen:latest", interactive=True)
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])
    
    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    #cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])
    cpp_run.click(execute_cpp, inputs=[cpp, model], outputs=[cpp_out])
    
ui.launch(inbrowser=False)